In [2]:
# Dependencies and Setup
from datetime import datetime
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [37]:
# Day of the week
today = datetime.today()
weekday = today.isoweekday()

days_week_dict = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday",
                 5: "Friday", 6: "Saturday", 7: "Sunday"}
day_of_the_week = days_week_dict.get(weekday)

print(day_of_the_week)

Monday


In [38]:
# Creates directory for current day of the week, raises an error if already exists.
parent_dir = 'json_files'
output_dir = f'{day_of_the_week}1000'
path = os.path.join(parent_dir, output_dir)

try:
    os.mkdir(path)
except:
    print('Folder already exists.')

Folder already exists.


In [39]:
# API Call
load_dotenv()
API_KEY = os.getenv("Yelp_API") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"

In [40]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204


In [41]:
# Make a list of coordinate tuples (lat, lng, city) for each team
stadium_zip = zip(stadium_data_df["Latitude"], stadium_data_df["Longitude"], stadium_data_df['NFL Team'])
stadium = list(stadium_zip)
#print(stadium))

In [42]:
# API Parameters
search_limit = 20 #limit to 20 business names per query
radius = 3000 #radius in meters
term = "restaurants"
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating"}

# lists to store the data retrieve from Yelp
business_name = []
business_state = []
business_price = []
business_rating = []
business_reviews = []
number_business = []
#print(stadium)

In [43]:
# Create Exception

class Exception_1(Exception):
    """ Raised if "total" is not found in the request """
    pass

In [47]:
#Loop to dump data into json files based on (lat,lng)
count_f = 0


for latitude, longitude, team in stadium:
        
    search_params.update({"latitude": latitude, "longitude": longitude})
    response = requests.get(url = base_url, params = search_params, headers = headers).json()
    try:
        
        if response['total']== False:
            raise Exception_1
        elif response['total'] >= 1000:
            total = 1000
        else:
            total = response['total']
        
        for search_offset in range(0, total, 20):
        
            search_params.update({"offset": search_offset})
            response2 = requests.get(url = base_url, params = search_params, headers = headers).json()
            file_name = f'yelp_response_{team}_{search_offset}.json'
            output_path = os.path.join(path, file_name)
            
            with open(output_path,'w', encoding = 'utf-8') as f:
                json.dump(response2, f, ensure_ascii=False, indent =4)
            f.close()
            
    except Exception_1 as e:
        print("Error: 'total' not found.\nPlease try again later.")
        print(e)
        
    print(f'{count_f}, {team} total: {total}')
    
    count_f += 1
        

0, Arizona Cardinals total: 94
1, Atlanta Falcons total: 704
2, Baltimore Ravens total: 666
3, Buffalo Bills total: 51
4, Carolina Panthers total: 441
5, Chicago Bears total: 1000
6, Cincinnati Bengals total: 459
7, Cleveland Browns total: 355
8, Dallas Cowboys total: 210
9, Denver Broncos total: 575
10, Detroit Lions total: 531
11, Green Bay Packers total: 120
12, Houston Texans total: 180
13, Indianapolis Colts total: 354
14, Jacksonville Jaguars total: 175
15, Kansas City Chiefs total: 18
16, Las Vegas Raiders total: 758
17, Los Angeles Chargers total: 293
18, Los Angeles Rams total: 293
19, Miami Dolphins  total: 84
20, Minnesota Vikings total: 682
21, New England Patriots total: 44
22, New Orleans Saints total: 945
23, New York Giants total: 133
24, New York Jets total: 133
25, Philadelphia Eagles total: 396
26, Pittsburgh Steelers total: 380
27, San Francisco 49ers total: 179
28, Seattle Seahawks total: 1000
29, Tampa Bay Buccaners total: 270
30, Tennessee Titans total: 528
31, W

## Check json files integrity
Sometimes, due to an internal error in Yelp, the YELP API would return an empty file with an error code in it. 

Print to terminal if the json file has errors or not. 

If there are files with errors, the above request data needs to be re-run.

In [5]:
# Check error in json files.

files_list = os.listdir(path)
remove_list = []
for file in files_list:
    open_path = os.path.join(path, file)
    with open(open_path, 'r', encoding = 'utf-8') as f:
        contents = json.loads(f.read())
        try:
            total = contents['total']
                        
        except:
            print(f"INTERNAL_ERROR in {file}")
            print("Something went wrong internally, please try downloading the json file again later.\n")
            file = f'{file}'
            remove_list.append(file)
        else:
            print(f'\nNo errors found in the json file:\n{file}\n')
        



No errors found in the json file:
yelp_response_Arizona Cardinals_0.json


No errors found in the json file:
yelp_response_Arizona Cardinals_20.json


No errors found in the json file:
yelp_response_Arizona Cardinals_40.json


No errors found in the json file:
yelp_response_Arizona Cardinals_60.json


No errors found in the json file:
yelp_response_Arizona Cardinals_80.json


No errors found in the json file:
yelp_response_Atlanta Falcons_0.json


No errors found in the json file:
yelp_response_Atlanta Falcons_100.json


No errors found in the json file:
yelp_response_Atlanta Falcons_120.json


No errors found in the json file:
yelp_response_Atlanta Falcons_140.json


No errors found in the json file:
yelp_response_Atlanta Falcons_160.json


No errors found in the json file:
yelp_response_Atlanta Falcons_180.json


No errors found in the json file:
yelp_response_Atlanta Falcons_20.json


No errors found in the json file:
yelp_response_Atlanta Falcons_200.json


No errors found in the 



No errors found in the json file:
yelp_response_Chicago Bears_580.json


No errors found in the json file:
yelp_response_Chicago Bears_60.json


No errors found in the json file:
yelp_response_Chicago Bears_600.json


No errors found in the json file:
yelp_response_Chicago Bears_620.json


No errors found in the json file:
yelp_response_Chicago Bears_640.json


No errors found in the json file:
yelp_response_Chicago Bears_660.json


No errors found in the json file:
yelp_response_Chicago Bears_680.json


No errors found in the json file:
yelp_response_Chicago Bears_700.json


No errors found in the json file:
yelp_response_Chicago Bears_720.json


No errors found in the json file:
yelp_response_Chicago Bears_740.json


No errors found in the json file:
yelp_response_Chicago Bears_760.json


No errors found in the json file:
yelp_response_Chicago Bears_780.json


No errors found in the json file:
yelp_response_Chicago Bears_80.json


No errors found in the json file:
yelp_response_Chi



No errors found in the json file:
yelp_response_Los Angeles Chargers_80.json


No errors found in the json file:
yelp_response_Los Angeles Rams_0.json


No errors found in the json file:
yelp_response_Los Angeles Rams_100.json


No errors found in the json file:
yelp_response_Los Angeles Rams_120.json


No errors found in the json file:
yelp_response_Los Angeles Rams_140.json


No errors found in the json file:
yelp_response_Los Angeles Rams_160.json


No errors found in the json file:
yelp_response_Los Angeles Rams_180.json


No errors found in the json file:
yelp_response_Los Angeles Rams_20.json


No errors found in the json file:
yelp_response_Los Angeles Rams_200.json


No errors found in the json file:
yelp_response_Los Angeles Rams_220.json


No errors found in the json file:
yelp_response_Los Angeles Rams_240.json


No errors found in the json file:
yelp_response_Los Angeles Rams_260.json


No errors found in the json file:
yelp_response_Los Angeles Rams_280.json


No errors 



No errors found in the json file:
yelp_response_Tennessee Titans_140.json


No errors found in the json file:
yelp_response_Tennessee Titans_160.json


No errors found in the json file:
yelp_response_Tennessee Titans_180.json


No errors found in the json file:
yelp_response_Tennessee Titans_20.json


No errors found in the json file:
yelp_response_Tennessee Titans_200.json


No errors found in the json file:
yelp_response_Tennessee Titans_220.json


No errors found in the json file:
yelp_response_Tennessee Titans_240.json


No errors found in the json file:
yelp_response_Tennessee Titans_260.json


No errors found in the json file:
yelp_response_Tennessee Titans_280.json


No errors found in the json file:
yelp_response_Tennessee Titans_300.json


No errors found in the json file:
yelp_response_Tennessee Titans_320.json


No errors found in the json file:
yelp_response_Tennessee Titans_340.json


No errors found in the json file:
yelp_response_Tennessee Titans_360.json


No errors f

In [6]:
if not remove_list:
    print(f'No erros in json files.\nNo files to be removed for the analysis.')
else:
    print(f'Files with errors:\n{remove_list}')

No erros in json files.
No files to be removed for the analysis.
